In [1]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.Pendulum import Pendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


In [2]:
# Load Data:
with jnp.load('Data/data_pods.npz') as data:
    X_data1 = data['states_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['states_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)
print(X_data[:10,:], y_data[:10,:])
rng = jax.random.PRNGKey(0)

(1782000, 5) (1782000, 4)
[[ 4.9494696   2.7237606   1.7805529   0.68680406 -0.9269727 ]
 [ 5.012753    2.744014    1.5158426   0.4451526  -0.89220834]
 [ 5.0657682   2.7551312   1.2619146   0.22220194 -0.8692082 ]
 [ 5.108873    2.7579658   1.0163066   0.02130936 -0.8460146 ]
 [ 5.142433    2.7536917   0.7801123  -0.14801113 -0.82660663]
 [ 5.1668115   2.7436981   0.55281645 -0.28176498 -0.80512494]
 [ 5.182396    2.7295787   0.3354695  -0.37448558 -0.78440076]
 [ 5.1895795   2.713002    0.12788771 -0.4254818  -0.7576858 ]
 [ 5.1888046   2.6957269  -0.06824264 -0.43198076 -0.7322167 ]
 [ 5.180503    2.6793983  -0.25379708 -0.39864138 -0.70209825]] [[ 5.012753    2.744014    1.5158426   0.4451526 ]
 [ 5.0657682   2.7551312   1.2619146   0.22220194]
 [ 5.108873    2.7579658   1.0163066   0.02130936]
 [ 5.142433    2.7536917   0.7801123  -0.14801113]
 [ 5.1668115   2.7436981   0.55281645 -0.28176498]
 [ 5.182396    2.7295787   0.3354695  -0.37448558]
 [ 5.1895795   2.713002    0.12788771

In [3]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.001, num_epochs=30, batch_size=64)

0
Epoch 0, Train Loss: 0.0016, Test Loss: 0.0016
1
Epoch 1, Train Loss: 0.0007, Test Loss: 0.0007
2
Epoch 2, Train Loss: 0.0005, Test Loss: 0.0005
3
Epoch 3, Train Loss: 0.0004, Test Loss: 0.0004
4
Epoch 4, Train Loss: 0.0003, Test Loss: 0.0003
5
Epoch 5, Train Loss: 0.0003, Test Loss: 0.0003
6
Epoch 6, Train Loss: 0.0004, Test Loss: 0.0004
7
Epoch 7, Train Loss: 0.0003, Test Loss: 0.0003
8
Epoch 8, Train Loss: 0.0003, Test Loss: 0.0003
9
Epoch 9, Train Loss: 0.0003, Test Loss: 0.0003
10
Epoch 10, Train Loss: 0.0003, Test Loss: 0.0003
11
Epoch 11, Train Loss: 0.0003, Test Loss: 0.0003
12
Epoch 12, Train Loss: 0.0003, Test Loss: 0.0003
13
Epoch 13, Train Loss: 0.0002, Test Loss: 0.0002
14
Epoch 14, Train Loss: 0.0002, Test Loss: 0.0002
15
Epoch 15, Train Loss: 0.0002, Test Loss: 0.0002
16
Epoch 16, Train Loss: 0.0003, Test Loss: 0.0003
17
Epoch 17, Train Loss: 0.0003, Test Loss: 0.0003
18
Epoch 18, Train Loss: 0.0002, Test Loss: 0.0002
19
Epoch 19, Train Loss: 0.0002, Test Loss: 0.0002


In [4]:
from env1.Pendulum import Pendulum as env1

env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
#print(State.pipeline_state)
#State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )
print(qqd10, qqd11)



11
[[-4.364456 ]
 [-3.306514 ]
 [ 2.0459676]
 [-0.8970223]] [-4.3660116 -3.3631048  2.0354965 -0.9495645]


In [6]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
